In [15]:
import numpy as np
import utils.optlib as opt
import plotly.graph_objects as go

In [16]:
# setting the underlying
stock = opt.Underlying(price=65,
                       volatility=0.3,
                       interest=0.1)

# creating the European Put option
EuPut = opt.Option(underlying=stock,
                   style="European",
                   call=False,
                   strike=50)

# creating the grid and adding option's parameters
grid = opt.Grid(xSteps=300,
                tSteps=200,
                xLeft=-3.0,
                xRight=3.0)

grid.addOption(EuPut)
print("Curant number", grid.lamda)
# setting boundary conditions

Curant number 0.5625


In [17]:
net = grid.net.copy()
q = grid.q
n = grid.xSteps
c = grid.lamda
# creating matrices for schemes
A = np.diag(np.ones(n+1) * (1 + c))
B = np.diag(np.ones(n+1) * (1 - c))
for i in range(1, n + 1):
    A[i, i-1] = -c/2
    A[i-1, i] = -c/2
    B[i, i-1] = c/2
    B[i-1, i] = c/2

### Forward $y(x, \tau)$ computation

In [18]:
# setting initial condition (without boundaries)
for i in np.arange(0, grid.xSteps):
    net[i, 0] = opt.g_func(q, grid.tGrid[0], grid.xGrid[i])

# crank-nickolson scheme
for i in np.arange(0, grid.tSteps):
    # explicit step
    f = np.dot(B, net[:, i].copy())
    f[0] = f[0] + 0.5 * c * (opt.g_func(q, grid.tGrid[i], grid.xGrid[0]) + opt.g_func(q, grid.tGrid[i+1], grid.xGrid[0]))
    # implicit step
    solution = opt.scalar_walk(A, f)
    net[:, i + 1] = solution

In [19]:
grid.net = net.copy()
grid.toNormal(obj_mutation=True)
grid.plot(0, cut=True)

### Adjoint $\lambda^{T}$ computation (backward loop)

In [20]:
N = grid.tSteps
M = grid.xSteps

lambda_vec = np.empty((M+1, N+1))
lambda_vec[:,N] = np.ones(M+1)

A_inv = np.linalg.inv(A)
f_y = np.dot(A_inv, B)

for i in range(N, 1, -1):
    lambda_vec[:, i-1] = np.dot(f_y, lambda_vec[:,i])

### Adjoint Vega calculation (forward loop)

In [21]:
lambda_sigma = stock.volatility * grid.option.maturity * (M+1)**2 / ((N+1) * (grid.xRight - grid.xLeft)**2)
A_sigma = np.diag(2 * np.ones(M+1))
for i in range(1, M + 1):
    A[i, i-1] = -1
    A[i-1, i] = -1
A_sigma = A_sigma * lambda_sigma / 2
B_sigma = -A_sigma

def d_sigma(ind):
    x = grid.xGrid[0]
    t1 = grid.tGrid[ind]
    t2 = grid.tGrid[ind+1]
    q = grid.q

    el1 = opt.g_func(q, t1, x) + opt.g_func(q, t2, x)
    el2 = t1 * opt.g_func(q, t1, x) + t2 * opt.g_func(q, t2, x)

    d1 = el1 * lambda_sigma / 2
    d2 = (- 2 * stock.interest / stock.volatility**3) * (el1 * x + (q+1) * el2)
    return d1 + d2


In [22]:
Vega = np.empty((M+1, N+1))

x_vec =  grid.xGrid.copy()
y0_sigma = np.multiply(net[:,0], -2 * x_vec * stock.interest / stock.volatility**3)
Vega[:,0] = np.multiply(lambda_vec[:,0], y0_sigma)

# Adjoint vector of linear system
lambda_linear = np.dot(A_inv, np.ones(M+1))
for i in range(1, N+1):
    # linear system adjoint solving (finding foo_sigma)
    d_sigma_vec = np.zeros(M+1)
    d_sigma_vec[0] = d_sigma(i-1)
    something = B_sigma @ net[:,i-1] + B @ Vega[:,i-1] + d_sigma_vec - A_sigma @ net[:,i]
    foo_sigma = np.multiply(lambda_linear, something)

    Vega[:,i] = Vega[:,i-1] + np.multiply(lambda_vec[:,i], foo_sigma)

In [25]:
grid.net = Vega.copy()
grid.plot(0, cut=True)

In [26]:
grid.toNormal(obj_mutation=True)
grid.plot(0, cut=True)

D:\Code\Python\noaMrFendel\docs\quant\utils\optlib.py:140: RuntimeWarning:

overflow encountered in exp



IndexError: index -1 is out of bounds for axis 0 with size 0